This program reads data from the Openflights.org website and stores them in a SQLite database.
It runs 3 simple queries and one join.

In [2]:
import sqlite3 as sql
import pandas as pd

In [3]:
conn = sql.connect('airline.db')
cursor = conn.cursor()

cursor.execute("Drop table if exists route")
cursor.execute("Drop table if exists airport")
cursor.execute("Drop table  if exists airline")

In [4]:
airportfile="airports.csv"
dfport = pd.read_csv(airportfile, index_col=None, encoding = "ISO-8859-1")
dfport.head()

,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081689834590001,145.391998291,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
0,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
2,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
3,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
4,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports


Routes columns
 - Airline	2-letter (IATA) or 3-letter (ICAO) code of the airline.
 - Airline ID	Unique OpenFlights identifier for airline (see Airline).
 - Source airport	3-letter (IATA) or 4-letter (ICAO) code of the source airport.
 - Source airport ID	Unique OpenFlights identifier for source airport (see Airport)
 - Destination airport	3-letter (IATA) or 4-letter (ICAO) code of the destination airport.
 - Destination airport ID	Unique OpenFlights identifier for destination airport (see Airport)
 - Codeshare	"Y" if this flight is a codeshare (that is, not operated by Airline, but another carrier), empty otherwise.
 - Stops	Number of stops on this flight ("0" for direct)
 - Equipment	3-letter codes for plane type(s) generally used on this flight, separated by spaces

In [5]:
#routefile = "routes.csv"
routefile = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"
headers = ["Code", "AirlineID", "OriginIATA", "OriginID","DestIATA", "DestID", 
           "Codeshare", "StopNums", "Equip"]
dfrout = pd.read_csv(routefile, index_col=None,names=headers, encoding='utf-8')
dfrout.head()

,Code,AirlineID,OriginIATA,OriginID,DestIATA,DestID,Codeshare,StopNums,Equip
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [6]:
airlinefile ="https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat"
headers = ["id", "AirlineName", "Alias", "IATA", "ICAO", "Callsign", "Country", "Active"]
dfline = pd.read_csv(airlinefile, index_col=None, names=headers, encoding='utf-8')
dfline.head()

,id,AirlineName,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N


In [7]:
dfline.to_sql('airline', conn)
dfport.to_sql('airport', conn)
dfrout.to_sql('route', conn)

/Users/abrahamizarra/Library/Python/3.8/lib/python/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [8]:
pd.read_sql_query("select * from airport limit 5",conn)

,index,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081689834590001,145.391998291,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
0,0,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
1,1,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
2,2,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
3,3,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
4,4,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports


In [9]:
pd.read_sql_query("select * from route limit 5",conn)

,index,Code,AirlineID,OriginIATA,OriginID,DestIATA,DestID,Codeshare,StopNums,Equip
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


In [10]:
pd.read_sql_query("select * from airline limit 5",conn)

,index,id,AirlineName,Alias,IATA,ICAO,Callsign,Country,Active
0,0,-1,Unknown,\N,-,None,\N,\N,Y
1,1,1,Private flight,\N,-,None,None,None,Y
2,2,2,135 Airways,\N,None,GNL,GENERAL,United States,N
3,3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N


In [11]:
pd.read_sql_query("select * from airline join route \
on airline.id = route.airlineid \
where lower(airlinename) like 'ryanair' limit 5", conn)

,index,id,AirlineName,Alias,IATA,ICAO,Callsign,Country,Active,index,Code,AirlineID,OriginIATA,OriginID,DestIATA,DestID,Codeshare,StopNums,Equip
0,4290,4296,Ryanair,\N,FR,RYR,RYANAIR,Ireland,Y,25797,FR,4296,AAR,607,AGP,1230,None,0,738
1,4290,4296,Ryanair,\N,FR,RYR,RYANAIR,Ireland,Y,25798,FR,4296,AAR,607,PMI,3998,None,0,738
2,4290,4296,Ryanair,\N,FR,RYR,RYANAIR,Ireland,Y,25799,FR,4296,AAR,607,STN,548,None,0,738
3,4290,4296,Ryanair,\N,FR,RYR,RYANAIR,Ireland,Y,25800,FR,4296,ACE,1055,BCN,1218,None,0,738
4,4290,4296,Ryanair,\N,FR,RYR,RYANAIR,Ireland,Y,25801,FR,4296,ACE,1055,BGY,1525,None,0,738


In [12]:
conn.close()